In [ ]:
st.set_page_config(layout="wide", page_title="Finance Agent")

# Global variable to store logs
if 'logs' not in st.session_state:
    st.session_state.logs = []

# Custom print function to capture output
def custom_print(*args, **kwargs):
    output = " ".join(map(str, args))
    st.session_state.logs.append(output)
    sys.__stdout__.write(output + "\n")

# Replace the built-in print function
print = custom_print

st.title("Finance Agent Query Interface")

# Sidebar for query input
with st.sidebar:
    st.header("Query Input")
    query = st.text_area("Enter your finance-related query:", height=100)
    run_button = st.button("Run Query", type="primary")

# Main content area
if run_button and query:
    # Clear previous logs
    st.session_state.logs = []

    # Create placeholders for live updates
    log_placeholder = st.empty()
    result_placeholder = st.empty()

    async def run_query():
        result = await run_finance_agent(query)
        return result

    # Run the query and update the UI
    with st.spinner("Processing your query..."):
        result = asyncio.run(run_query())

        # Display the final result
        with result_placeholder.container():
            st.success("Query processing completed!")
            
            # Display graph database results if available
            if 'graph_db_result' in result:
                st.subheader("Graph Database Results")
                st.json(result['graph_db_result'])
            
            # Display expanded queries if available
            if 'expanded_queries' in result:
                st.subheader("Expanded Queries")
                for i, eq in enumerate(result['expanded_queries'], 1):
                    st.write(f"{i}. {eq}")
            
            # Display subgraph results if available
            if 'subgraph_results' in result:
                st.subheader("Subgraph Results")
                for subgraph, subresult in result['subgraph_results'].items():
                    with st.expander(f"Results from {subgraph}"):
                        st.json(subresult)
            
            # Display final response
            st.subheader("Final Response")
            st.write(result.get('final_response', 'No final response available.'))

    # Display logs
    with log_placeholder.container():
        st.subheader("Process Log")
        for log in st.session_state.logs:
            st.text(log)

else:
    st.info("Enter a query in the sidebar and click 'Run Query' to start processing.")

# Footer
st.markdown("---")
st.markdown("© 2023 Finance Agent. All rights reserved.")